# Training Deep Neural Networks

**Vanishing / Exploding Gradients Problem:** 
When the gradients become too big / too small to be propagated through the NN.
Solution: Use a BatchNormalisation after each layer, except the output.



In [ ]:
model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=[28, 28]),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(10, activation="softmax")
])

**WAYS TO SPEED UP TRAINING**:

- Good initialisation strategy for connection weights
- Good activation function
- Batch normalisation / Gradient Clipping
- Reuse parts of a pre-trained network
- Faster optimisers

## Function & Activation
So which activation function should you use for the hidden layers of your deep neural networks? 

In general **SELU > ELU > leaky ReLU (and its variants) > ReLU > tanh > logistic.** 
- If the network’s architecture prevents it from self- normalizing, then ELU may perform better than SELU (since SELU is not smooth at z = 0). 
- If you care a lot about runtime latency, then you may prefer leaky ReLU. If you don’t want to tweak yet another hyperparameter, you may just use the default α values used by Keras (e.g., 0.3 for the leaky ReLU). 
- If you have spare time and computing power, you can use cross-validation to evaluate other activation functions, in particular RReLU if your network is over‐ fitting, or PReLU if you have a huge training set.

- To use the leaky ReLU activation function, you must create a LeakyReLU instance like this:

> leaky_relu = keras.layers.LeakyReLU(alpha=0.2)

> layer = keras.layers.Dense(10, activation=leaky_relu, kernel_initializer="he_normal")

- For PReLU, just replace LeakyRelu(alpha=0.2) with PReLU().

- For SELU activation, just set activation="selu" and kernel_initial izer="lecun_normal" when creating a layer:
> layer = keras.layers.Dense(10, activation="selu", kernel_initializer="lecun_normal")

## Gradient Clipping

To solve the issue of exploding gradients in RNNs, it's better to use Gradient Clipping because the implementation of Batch Normalisation is more tricky.
To do so, just use the clipvalue when creating an optimiser:

> optimiser = keras.optimizer.SGD(clipvalue=1.0)  #clip every component of the gradient vector to a value between –1.0 and 1.0


You can also clip the norm, instead of the value, if you notice the gradients explode (look for more info on how to do that).

## Faster Optimizers

**Momentum optimization:** you just have to set the parameter *momentum* (0.9 is usually a good value)
>> optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9) 

**Nesterov Accelerated Gradient** sometimes improves performance. It's a variation of momentum optimization. To use set *nesterov=True*
>> optmizer = keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)

**AdaGrad** this algorithm decays the learning rate, but it does it faster for steep dimensions than for dimensions with deeper slopes (known as *adaptative learning rate*).  It doesn't work well for NNs, but it's sufficient for Linear Regression.

**RMSProp** it's a variation of AdaGrad, but only accumulates gradients of recent iterations, so it performs better. The default valud of rho works well.

>> optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9)

**Adam** (adaptative moment estimation) it's a variation of Momentum optimisation and RMSProp. The default values work well because it's an adaptive learning algorithm.

>> optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)


